In [1]:
version = "REPLACE_PACKAGE_VERSION"

---
# Assignment 3: Adaptive Information Filtering (50 pts)

In the previous two assignments, we have been building information retrieval systems that address a user's ad-hoc information needs expressed in a set of queries. Now we will attend to a user's long-term information needs by actively "pushing" information of interest to the user through Adaptive Information Filtering. Let's pretend we are running a newsfeed business that delivers [BBC news](http://mlg.ucd.ie/datasets/bbc.html) of five topics (business, entertainment, politics, sport and tech) to subscribers. Our goal is to infer a subscriber's interest from the feedback we receive and to deliver news of the right topics. 

Below is the definition of the `BBCNewsfeed` class that we will use to model a newsfeed. It is very similar to the `TwitterStream` class you worked with in Assignment 4 of `SIADS 632 Data Mining II`. Both classes are implemented as a Python `iterable` that can be used in a `for`-loop. You may refer to Assignment 4 of 632 for more examples about working with `iterable`s. At each iteration, an instance of this iterable `BBCNewsfeed` class returns

* a row vector in the form of a [CSR sparse matrix](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.html) of shape `(1, 9635)` representing a news article; and 

* an integer from `0-4` (inclusive) representing the topic of the news article. 

A mapping from integers to topics is given as the class variable `ids_to_topics`. An example of retrieving the first news article is also provided. 

In [2]:
import os
import math
import random
import scipy.io
import numpy as np


class BBCNewsfeed:
    
    ids_to_topics = {0: "business", 1: "entertainment", 2: "politics", 3: "sport", 4: "tech"}

    def __init__(self, data_path, random_state=None):
        self._random = random.Random(random_state)
        
        # load data
        tf = scipy.io.mmread(os.path.join(data_path, "bbc.mtx")).T.tocsr()  # (n_docs, n_terms)
        df = np.nansum(tf / tf, axis=0)
        self._data = np.log1p(tf).multiply(1 + math.log(tf.shape[0]) - np.log(df)).tocsr() # tf-idf
        self.shape = self._data.shape
        
        # load labels
        classes = np.genfromtxt(os.path.join(data_path, "bbc.classes"), dtype=int, comments="%")
        self._classes = classes[self._random.sample(range(classes.shape[0]), classes.shape[0])]
        
        self._curr_row = 0

    def __iter__(self):
        return self.reset()
    
    def __next__(self):
        if self._curr_row < self._classes.shape[0]:
            idx, cls = self._classes[self._curr_row]
            self._curr_row += 1
            return self._data[idx], cls
        else:
            raise StopIteration
    
    def reset(self):
        self._curr_row = 0
        return self

In [3]:
# Grab the first news article
bbc_newsfeed = BBCNewsfeed("assets/bbc", random_state=42)
next(bbc_newsfeed) # returns a tuple

(<1x9635 sparse matrix of type '<class 'numpy.float64'>'
 	with 122 stored elements in Compressed Sparse Row format>,
 0)

As discussed in the lecture, we can build an adaptive information filter using a binary classifier that determines whether a news article is of interest to the subscriber. After we recommend some news articles to the subscriber, the classifier is then trained on the subscriber's relevance feedback on our recommendations. The same back-and-forth interactions repeat until our newsfeed business goes bankrupt with no more news articles to deliver. 

Your task for this assignment is to complete the `AdapInfoFilter` class below that implements the said adaptive information filter. More specifically, you will need to complete the following two methods which act sequentially during each round of interaction.

* **`do_filtering`**: This method takes in a `newsfeed` as defined above and steps into the newsfeed to receive the vector representation of a news article, `news_vec`, and the topic of the news article, `topic`, at every iteration. Whenever it has amassed a batch of `batch_sz` news articles and topics, it should use its classifier `self.clf` to select news articles of interest and the corresponding topics, and `yield` them as a `tuple` (e.g., `yield news_vecs, topics`, where `news_vecs` is a 2D CSR sparse matrix of shape `(N, 9635)` and `topics` is a `np.ndarray` of shape `(N,)`. And `N` is dynamically determined by the classifier. ) This process bears some resemblance to the `LossyCounter` you have implemented in 632, where you perform lossy counting only after a full *bucket* of tweets have arrived. It is OK if your `clf` selects zero news articles, in which case you will just `yield` a degenerate 2D CSR sparse matrix of shape `(0, 9635)` and a degenerate `np.ndarray` of shape `(0,)`; as a result, though, you won't receive any feedback during that round of interaction. 


* **`gather_feedback`**: The `feedback` from the subscriber is passed to this method as a binary `np.ndarray` of shape `(N,)` with `1` indicating relevance and `0` otherwise, along with the news vectors `news_vecs` and the topics `topics` that were `yield`-ed during the current round of interaction. You should then train your `clf` *incrementally* using `feedback`. You may use any classifier of your choice, although only a few classifiers in `sklearn` support the `partial_fit` method that allows online, incremental learning. It is also possible to append `topics` as an extra column to `news_vecs` to create new training data. 


You may have wondered, if the very first batch of news articles are selected *before* your `clf` ever receives any feedback to start learning, how the first batch should be selected. Our solution to this "chicken-or-egg" dilemma is that **we don't do any filtering on the first batch** --- we let it go in order to receive some initial feedback; but from the second batch onwards your `clf` should start to do its job. This is equivalent to providing you with a few initial relevant news articles to initialise your `clf`, a way of dealing with cold start. 

In [4]:
from sklearn.linear_model import SGDClassifier, PassiveAggressiveClassifier, Perceptron


class AdapInfoFilter:
    
    def __init__(self, random_state=None):
        # Initialize with None, will be created after first batch
        self.clf = None
        self.random_state = random_state
    
    def do_filtering(self, newsfeed, batch_sz=100):
        news_vecs, topics = [], []
        for idx, (news_vec,topic) in enumerate(newsfeed):
            news_vecs.append(news_vec)
            topics.append(topic)
            
            if len(news_vecs) == batch_sz:
                news_vecs_array = scipy.sparse.vstack(news_vecs)
                topics_array = np.array(topics)
                
                if self.clf:
                    predicted_interest = self.clf.predict(news_vecs_array)
                    selected_news = news_vecs_array[predicted_interest == 1]
                    selected_topics = topics_array[predicted_interest == 1]
                    yield selected_news, selected_topics
                else:
                    yield news_vecs_array, topics_array
                news_vecs, topics = [], []


    def gather_feedback(self, feedback, news_vecs, topics):
        if feedback.size > 0 & news_vecs.shape[0] > 0:
            if self.clf is None:
                self.clf = PassiveAggressiveClassifier(loss='log', random_state=self.random_state)
                self.clf.partial_fit(news_vecs, feedback, classes=np.array([0, 1]))
            else:
                self.clf.partial_fit(news_vecs, feedback,classes=np.array([0, 1]))


Behind the scene is there a `Subscriber` class whose definition is hidden. But after an ordinary `import`, you can create an instance of the `Subscriber` class to play with. The `give_feedback` method of the `Subscriber` class is the most relevant to you, from which your `AdapInfoFilter` will receive feedback. An illustration of how an `Subscriber` interacts with an `AdapInfoFilter` is provided in the visible tests below. You don't need to worry too much about the details of the function calls, as long as your `do_filtering` and `gather_feedback` methods do what they are supposed to do. 

In [5]:
from utils import Subscriber

# create a new instance to play with
subscriber = Subscriber(random_state=42)

In [6]:
# The definition of Subscriber is hidden here - 0 pts


In [7]:

# Autograder tests - check do_filtering and gather_feedback (30 pts)
import scipy.sparse
import numpy as np

# These won't change in the hidden tests
random_state, batch_sz = 42, 100

bbc_newsfeed = BBCNewsfeed("assets/bbc", random_state)
stu_filter = AdapInfoFilter(random_state)

subscriber = Subscriber(random_state=random_state)

# Some sanity checks
count = 0
for idx, stu_batch in enumerate(stu_filter.do_filtering(bbc_newsfeed, batch_sz=batch_sz)):
    
    count += 1
    
    assert isinstance(stu_batch, tuple), f"Q1: For batch at index {idx}, your 'do_filtering' function should yield a tuple. "
    assert len(stu_batch) == 2, f"Q1: For batch at index {idx}, the tuple yielded is of an incorrect length. "
    
    # check news_vecs
    assert scipy.sparse.isspmatrix_csr(stu_batch[0]) or isinstance(stu_batch[0], np.ndarray), f"Q1: For batch at index {idx}, your news_vecs must be either a csr_matrix or a ndarray. "
    assert stu_batch[0].ndim == 2, f"Q1: For batch at index {idx}, your news_vecs must be a 2D array (sparse or dense). "
    assert stu_batch[0].shape[0] <= batch_sz, f"Q1: For batch at index {idx}, your news_vecs has more rows than batch_sz = {batch_sz}. "
    assert stu_batch[0].shape[1] == bbc_newsfeed.shape[1], f"Q1: For batch at index {idx}, your news_vecs has an incorrect # columns. "

    # check topics
    assert isinstance(stu_batch[1], np.ndarray), f"Q1: For batch at index {idx}, your topics must be a np.ndarray. "
    assert stu_batch[1].ndim == 1, f"Q1: For batch at index {idx}, your topics must be a 1D np.ndarray. "
    assert stu_batch[1].shape[0] <= batch_sz, f"Q1: For batch at index {idx}, your topics has more elements than batch_sz = {batch_sz}. "

    assert stu_batch[0].shape[0] == stu_batch[1].shape[0], f"Q1: For batch at index {idx}, the # rows of your news_vecs should equal to the # elements in your topics. "
    
    ######################################
    # The real interactions happen below #
    ######################################
    
    # Step 1: Your filter recommends some news articles and topics
    stu_news_vecs, stu_topics = stu_batch
    
    # Step 2: The subscriber gives feedback on the same news articles 
    feedback = subscriber.give_feedback(stu_news_vecs, stu_topics)
    
    # Step 3: Your filter gathers the feedback and updates its clf
    stu_filter.gather_feedback(feedback, stu_news_vecs, stu_topics)


assert count >= bbc_newsfeed.shape[0] // batch_sz, f"Q1: Your do_filtering function should at least yield {bbc_newsfeed.shape[0] // batch_sz} times. "


# Re-define variables to be used in hidden tests
bbc_newsfeed = BBCNewsfeed("assets/bbc", random_state)
stu_filter = AdapInfoFilter(random_state)
# A different subscriber will be used

# Some hidden tests

del random_state, batch_sz, bbc_newsfeed, subscriber, stu_filter

Your `AdapInfoFilter` will also be graded on the *utility* it achieves during the entire course of interaction. Whenever a recommended news article receives a 👍 from the subscriber, your `AdapInfoFilter` gets a utility of `+3`; otherwise, it gets a utility of `-1` (to prevent your `AdapInfoFilter` from overwhelming the subscriber with all the news articles indiscriminately). 

In [8]:
# Autograder tests - check if utility >= -40 (10 pts)

# These won't change in the hidden tests
random_state, batch_sz = 42, 100
util_pos, util_neg = 3, -1

bbc_newsfeed = BBCNewsfeed("assets/bbc", random_state)
stu_filter = AdapInfoFilter(random_state)

# Some hidden tests

In [9]:
# Autograder tests - check if utility >= -20 (10 pts)

# These won't change in the hidden tests
random_state, batch_sz = 42, 100
util_pos, util_neg = 3, -1

bbc_newsfeed = BBCNewsfeed("assets/bbc", random_state)
stu_filter = AdapInfoFilter(random_state)

# Some hidden tests

## References for data used

Greene, D., & Cunningham, P. (2006). Practical Solutions to the Problem of Diagonal Dominance in Kernel Document Clustering. In Proceedings of the 23rd International Conference on Machine Learning (pp. 377–384). Association for Computing Machinery.